# Personalized Real estate Agent

### AIML STREAM 
#### Name: Sneha Chakrabartty

This project utilizes the fundamentals of GenAI and it's use in the industry as a personalized recommendation system. This project is inclined towards fulfillment of the (Udacity nanodegree Final Project) AI/ML stream (Accenture Track). It demonstrates how Large Language Models (LLMs), vector databases, and Retrieval-Augumented Generation (RAG) can be combined to build a `Personalized Real Estate Recommendation Assistant`.

In this project the system does the following processes:
 - Synthetic data Generation: In this step the real estate listings are synthetically generated using LLM (Openai-3.5-Turbo) using a structured templates
 - Semantic Search: After generating synthetic data for real estate listings we store it into a vector database so that we can perform similarity search based on user preferences.
 - Augumented Response Generation: Once the synthetic listings are stored inside a vector database, the next step is to allow the system to retrieve the most relevant homes based on the buyer's preferences. This is done using RAG (Retrieval-Augumented Generation)
 - After this the most relevant lisitngs have been retrieved using semantic search, the next step is to personalize each listing so the output feels tailored uniquely to the buyer. This is done by utilizing `Conversational buffer memory` and `RAG-based contextual response generation`.
 
**Project Objectives and Tasks:**
Imagine you're a talented developer at "Future Homes Realty", a forward-thinking real estate company. In an industry where personalization is key to customer satisfaction, your company wants to revolutionize how clients interact with real estate listings. The goal is to create a personalized experience for each buyer, making the property search process more engaging and tailored to individual preferences.

**The Challenge**
Your task is to develop an innovative application named "HomeMatch". This application leverages large language models (LLMs) and vector databases to transform standard real estate listings into personalized narratives that resonate with potential buyers' unique preferences and needs.

**Core Components of "HomeMatch"**

**Understanding Buyer Preferences:**
Buyers will input their requirements and preferences, such as location, property type, budget, amenities, and lifestyle choices.
The application uses LLMs to interpret these inputs in natural language, understanding nuanced requests beyond basic filters.
Integrating with a Vector Database:

Connect "HomeMatch" with a vector database, where all available property listings are stored.
Utilize vector embeddings to match properties with buyer preferences, focusing on aspects like neighborhood vibes, architectural styles, and proximity to specific amenities.

**Personalized Listing Description Generation:**
For each matched listing, use an LLM to rewrite the description in a way that highlights aspects most relevant to the buyer’s preferences.
Ensure personalization emphasizes characteristics appealing to the buyer without altering factual information about the property.

**Listing Presentation:**
Output the personalized listing(s) as a text description of the listing.

In [107]:
!pip show openai

Name: openai
Version: 0.28.1
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [108]:
!pip show chromadb

Name: chromadb
Version: 0.4.12
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: bcrypt, chroma-hnswlib, fastapi, importlib-resources, numpy, onnxruntime, overrides, posthog, pulsar-client, pydantic, pypika, requests, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [109]:
!pip show langchain

Name: langchain
Version: 0.0.305
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Step 1: Setting Up the Python Application
Initialize a Python Project: Create a new Python project, setting up a virtual environment and installing necessary packages like LangChain, a suitable LLM library (e.g., OpenAI's GPT), and a vector database package compatible with Python (e.g., ChromaDB or LanceDB). If you don't wish to create your files from scratch, starter files are available in the workspace on the next page as an application skeleton.

In [110]:
# Importing all the libraries
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
import openai
import os

In [111]:
api_key = "voc-115557059912667743484506901cf4de79e13.88877519"
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = api_key

In [112]:
data_listings = [
    {'neighborhood':'Brixton', 'price': 225000, 'bedroom': 1, 'bathroom': 1, 'size': 850 },
    {'neighborhood':'Wimbledon', 'price': 320000, 'bedroom': 2, 'bathroom': 1, 'size': 1050 },
    {'neighborhood':'Twickenham', 'price': 450000, 'bedroom': 3, 'bathroom': 2, 'size': 1550 },
    {'neighborhood':'Canary Wharf', 'price': 320000, 'bedroom': 2, 'bathroom': 2, 'size': 1250 },
    {'neighborhood':'Hammersmith', 'price': 390000, 'bedroom': 3, 'bathroom': 1, 'size': 1450 },
    {'neighborhood':'Camden Town', 'price': 295000, 'bedroom': 1, 'bathroom': 1, 'size': 975 },
    {'neighborhood':'Richmond', 'price': 505000, 'bedroom': 3, 'bathroom': 2, 'size': 1750 },
    {'neighborhood':'Notting Hill', 'price': 650000, 'bedroom': 4, 'bathroom': 2, 'size': 2025 },
    {'neighborhood':'Covent Garden', 'price': 425000, 'bedroom': 2, 'bathroom': 1, 'size': 1250 },
    {'neighborhood':'Ealing', 'price': 350000, 'bedroom': 2, 'bathroom': 1, 'size': 1150 }    
]

In [113]:
prompt_template = """
    Generate real estate listings using a Large Language Model. Generate at least 10 listings that should contain neighborhood, price, number of bedrooms, number of bathrooms and house size (in square feet). The listing should contain the description and a neighborhood description.
    
    The real estate listing should use the following values 
    
    Neighborhood: {neighborhood}
    Price: ${price}
    Bedrooms: {bedroom}
    House Size: {size} sqft
    
    An example of a real estate listing is provided below:
    Neighborhood: Green Oaks
    Price: $800,000
    Bedrooms: 3
    Bathrooms: 2
    House Size: 2,000 sqft

    Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

    Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
    
    
"""

## Step 2: Generating Real Estate Listings
Generate real estate listings using a Large Language Model. Generate at least 10 listings This can involve creating prompts for the LLM to produce descriptions of various properties. 

In [114]:
def generate_real_estate_listing(prompt_template):
    try:
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": "You are a real estate agent. You are writing about real estate listing. "
          },
          {
            "role": "user",
            "content": prompt_template
          }
          ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [115]:
# Generating the response from the model
listings = []
count = 0
for s in data_listings:
    prompt = prompt_template.format(neighborhood=s['neighborhood'], price=s['price'], bedroom=s['bedroom'], bathroom=s['bathroom'], size=s['size'])
    listing = generate_real_estate_listing(prompt)
    listings.append("\n" + listing + "\n")
    file1 = open(f'data/listing{count}.txt', 'w')
    file1.write(listing)
    file1.close()
    count += 1
print("Saved the real estate listings")

Saved the real estate listings


In [117]:
file2 = open('Listings.txt', 'w')
file2.writelines(listings)
file2.close()

In [118]:
llm = ChatOpenAI()

In [119]:
loader = DirectoryLoader('./data', glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()

In [120]:
len(docs)

10

In [121]:
print(docs[0].page_content)

Neighborhood: Richmond
Price: $505,000
Bedrooms: 3
Bathrooms: 2
House Size: 1,750 sqft

Description: Step into this charming 3-bedroom, 2-bathroom home located in the desirable Richmond neighborhood. The house features a cozy living room, a well-appointed kitchen with stainless steel appliances, and a private backyard perfect for outdoor gatherings. With ample natural light, hardwood floors, and a spacious layout, this home is ideal for a growing family or those looking to downsize without compromising on comfort.

Neighborhood Description: Richmond is a vibrant neighborhood known for its blend of historic charm and modern convenience. Enjoy easy access to local shops, restaurants, and parks. Explore the nearby Richmond Art District or take a leisurely walk along the scenic riverfront. With top-rated schools and a strong sense of community, Richmond offers a welcoming atmosphere for residents of all ages.

Neighborhood: Richmond Estate
Price: $510,000
Bedrooms: 4
Bathrooms: 3
House Siz

## Step 3: Storing Listings in a Vector Database

Vector Database Setup: Initialize and configure ChromaDB or a similar vector database to store real estate listings.
Generating and Storing Embeddings: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.

In [122]:
# Use class CharacterTextSplitter to split the documents into chunks.
# This stores the embeddings in a more effective way.
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [123]:
# Initialize the embeddings model
embeddings = OpenAIEmbeddings()

In [124]:
# Populate the Chroma vector database with the chunks
db = Chroma.from_documents(split_docs, embeddings)

## Step 4: Building the User Preference Interface

Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language. 
Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database

In [125]:
# List of questions viewed as most helpful for the AI recommender
questions = [
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]
# List of your answers to the questions above
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [126]:
# Create a query based on the personal questions and answers
query = f"""
    {questions[0]} {answers[0]} 
    {questions[1]} {answers[1]}
    {questions[2]} {answers[2]}
    {questions[3]} {answers[3]}
    {questions[4]} {answers[4]}
    """

## Step 5: Searching Based on Preferences

**Semantic Search Implementation:** Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.

**Listing Retrieval Logic:** Fine-tune the retrieval algorithm to ensure that the most relevant listings are selected based on the semantic closeness to the buyer’s preferences.

In [127]:
results = db.similarity_search(query, k=5)
print(results)

[Document(page_content='1. **Neighborhood:** Twickenham\n   **Price:** E450,000\n   **Bedrooms:** 3\n   **Bathrooms:** 2\n   **House Size:** 1,550 sqft\n\n   **Description:** Step into this elegant 3-bedroom, 2-bathroom home located in the desirable Twickenham neighborhood. This property features a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a master suite with a luxurious en suite bathroom. Enjoy entertaining in the landscaped backyard with a charming patio area perfect for summer gatherings. The meticulous attention to detail and stylish finishes make this a must-see home in Twickenham.', metadata={'source': 'data/listing2.txt'}), Document(page_content='1. **Neighborhood:** Wimbledon\n**Price:** $320,000\n**Bedrooms:** 2\n**Bathrooms:** 1\n**House Size:** 1,050 sqft\n\n**Description:** Step into this cozy 2-bedroom, 1-bathroom home in the sought-after neighborhood of Wimbledon. The living room features a fireplace, perfect for coz

In [128]:
# Query the LLM with the query and the top 5 documents
use_chain_helper = True
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    # similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=results, query = query))

Based on your preferences, the property in the neighborhood of Wimbledon with 3 bedrooms, 2 bathrooms, a cozy living room, and a spacious kitchen would be a good fit for you. It is in a quiet neighborhood with good local schools and convenient shopping options. While the property doesn't meet all your desired amenities like a two-car garage and a modern, energy-efficient heating system, it does offer a spacious backyard for gardening. The neighborhood is described as tranquil with tree-lined streets and access to local boutiques and cafes. In terms of transportation options, it may not have easy access to a reliable bus line, but the neighborhood is bike-friendly and offers a balance between suburban tranquility and urban amenities.


## Step 6: Personalizing Listing Descriptions

**LLM Augmentation:** For each retrieved listing, use the LLM to augment the description, tailoring it to resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.

**Maintaining Factual Integrity:** Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

**ConversationBufferMemory:**

The ConversationBufferMemory component is essential for maintaining a history of chat messages, enabling the chatbot to have a contextual understanding of the ongoing conversation. When a user sends a message, it is added to the conversation history, and similarly, the chatbot's responses are also recorded. This helps in providing context for the chatbot's subsequent messages.
The conversation buffer memory can output the chat history either as a continuous stream or as a list.
The stored string of chat messages provides essential context for the language model (LM) when generating its next message.

In [132]:
history = ChatMessageHistory()
history.add_user_message(f"""You are an AI that will provide a real estate listing based on answers to the questions. Ask user {len(questions)} questions""")
# Add questions and answers to the history
for i in range(len(questions)):
    history.add_ai_message(questions[i])
    history.add_user_message(answers[i])    

In [133]:
# Create a memory that will have a summary of the recommendations
max_rating = 100

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary", 
    input_key="input",
    buffer=f"The human answered {len(questions)} questions). Use them to augment the real estate listing, tailoring it to resonate with the buyer’s specific preferences.",
    return_messages=True)

class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
    
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)

# Combined
memory = CombinedMemory(memories=[conversational_memory, summary_memory])

In [134]:
RECOMMENDER_TEMPLATE = """
The following is a friendly conversation between a human and an AI real estate agent. The AI follows human instructions
and provides an augmented real estate listing for a human based on the real estate listing provided. 

Summary of Recommendations:
{recommendation_summary}
Q&A:
{questions_and_answers}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
# Create a recommendation conversation chain that will let us ask AI for each retrieved listing
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)

In [135]:
# max_rating = 100
count = 1
for listing in listings:
    print("-----------------")
    print("-----------------")
    print("Listing number: " + str(count)) 
    print("-----------------")
    print("-----------------")
    listing_instructions = f"""
        \n\n\n\n\n\n\t\t\t\t\t HOME MATCH REAL ESTATE LISTING \t\t\t\t\t\n\n\n\n\n\n
        {listing}
        _____________________________________________________________________________________________________________
        
        LISTING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a real estate listing based only on the listing provided and human answers to questions included with the context. 
        AI should be very sensible to human personal preferences captured in the answers to personal questions and should not be influenced 
        by anything else. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for. 
        Ensure that the augmentation process enhances the appeal of the real estate listing without altering factual information.
        AI will also build a persona for human based on human answers to questions.
        OUTPUT FORMAT:
        It is very important to follow that same format as the real estate listing.
        Add the persona you came up with to the end of the real estate listing. Describe the persona in a few sentences.
        Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
        FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO UNDERSTAND YOUR REVIEW.
    """
    # Run the the recommendation chain to get a rating for the movie that will be summarized in the conversation summary
    prediction = recommender.predict(input=listing_instructions)
    print(prediction)
    count += 1

-----------------
-----------------
Listing number: 1
-----------------
-----------------


> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI real estate agent. The AI follows human instructions
and provides an augmented real estate listing for a human based on the real estate listing provided. 

Summary of Recommendations:
[SystemMessage(content='The human answered 5 questions). Use them to augment the real estate listing, tailoring it to resonate with the buyer’s specific preferences.')]
Q&A:
Human: You are an AI that will provide a real estate listing based on answers to the questions. Ask user 5 questions
AI: How big do you want your house to be?
Human: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
AI: What are 3 most important things for you in choosing this property?
Human: A quiet neighborhood, good local schools, and convenient shopping options.
AI: Which ame


> Finished chain.
**Listing 1 Augmented:**
   - Neighborhood: Wimbledon
   - Price: $320,000
   - Bedrooms: 2
   - Bathrooms: 1
   - House Size: 1,050 sqft

   **Description:** Welcome to this cozy 2-bedroom, 1-bathroom home nestled in the tranquil neighborhood of Wimbledon. The living room features a fireplace, perfect for those cozy evenings, and the kitchen is equipped with stainless steel appliances for your culinary needs. Step outside to enjoy the privacy of the spacious backyard, ideal for outdoor gatherings and relaxation. This charming home is a peaceful retreat for a small family or first-time homebuyers.

   **Neighborhood Description:** Wimbledon is known for its serene tree-lined streets and family-friendly atmosphere. Residents can take advantage of the nearby Wimbledon Park for outdoor picnics and leisurely walks. Explore the mix of trendy boutiques and cafes along the main street, offering a variety of shopping and dining options. With its convenient location near scho


> Finished chain.
**Listing Augmented:**
- Neighborhood: Twickenham
- Price: $475,000
- Bedrooms: 4
- Bathrooms: 3
- House Size: 1,550 sqft

**Description:** Welcome to this elegant 4-bedroom, 3-bathroom home in the desirable neighborhood of Twickenham. Step into the meticulously maintained property with a spacious floor plan and abundant natural light, highlighting the gleaming hardwood floors and modern finishes throughout. The chef's kitchen, equipped with stainless steel appliances and granite countertops, is perfect for entertaining guests. Retreat to the master suite for a spa-like experience in the ensuite bathroom. The private backyard oasis is ideal for outdoor gatherings, offering luxury living at its finest in Twickenham.

**Neighborhood Description:** Twickenham exudes historic charm and upscale ambiance, boasting a mix of stately homes and vibrant local amenities. Residents can enjoy easy access to boutique shops, gourmet restaurants, and cultural attractions. Take a leis


> Finished chain.
**Listing Augmented:**
- Neighborhood: Canary Wharf
- Price: $320,000
- Bedrooms: 2
- Bathrooms: 2
- House Size: 1,250 sqft

**Description:** Welcome to this modern 2-bedroom, 2-bathroom apartment in the vibrant neighborhood of Canary Wharf. This well-appointed unit features a spacious open-plan living area, a stylish kitchen with sleek appliances, and a private balcony offering stunning views of the city skyline. The bedrooms are generously sized with ample closet space, and the bathrooms are elegantly designed for a luxurious feel. Enjoy the convenience of nearby shopping, dining, and entertainment options, making this property a perfect urban retreat.

**Neighborhood Description:** Canary Wharf is a bustling financial district located along the Thames River, known for its iconic skyscrapers, upscale shops, and trendy restaurants. This vibrant area offers easy access to transportation links, including the Jubilee Line and DLR, making commuting a breeze. Residents c


> Finished chain.
**Listing Augmented:**
- Neighborhood: Hammersmith
- Price: $390,000
- Bedrooms: 3
- Bathrooms: 2
- House Size: 1,450 sqft

**Description:** Welcome to this stylish 3-bedroom, 2-bathroom home in the vibrant neighborhood of Hammersmith. The modern kitchen is equipped with stainless steel appliances and granite countertops, perfect for hosting gatherings. The cozy living room features a fireplace and abundant natural light, creating a warm and inviting atmosphere. Retreat to the spacious master bedroom with an ensuite bathroom for ultimate relaxation. Step outside to the private backyard oasis, ideal for summer BBQs and evening gatherings, offering the perfect blend of comfort and convenience in the heart of Hammersmith.

**Neighborhood Description:** Hammersmith is a vibrant and family-friendly neighborhood known for its tree-lined streets and community parks. Residents can explore local cafes, boutique shops, and farmers' markets, immersing themselves in the cultural


> Finished chain.
**Listing Augmented:**

- Neighborhood: Camden Town
- Price: $380,000
- Bedrooms: 2
- Bathrooms: 2
- House Size: 1200 sqft

**Description:** Discover this spacious 2-bedroom, 2-bathroom townhouse in the heart of vibrant Camden Town. This modern home features a sleek kitchen with stainless steel appliances, a cozy living area, and a private balcony offering stunning city views. The bedrooms boast ample closet space, and the bathrooms are elegantly designed. With hardwood floors throughout, this chic townhouse is perfect for urban living at its best.

**Persona:** The buyer persona for this augmented listing is someone who values modern urban living with stylish amenities. They appreciate the sleek kitchen, cozy living area, and private balcony with city views, aligning with their preferences for a chic and modern urban home. The buyer seeks convenience and style in their living space, which influenced the creation of this persona.
-----------------
-----------------
L


> Finished chain.
**Listing Augmented:**

- Neighborhood: Richmond
- Price: $505,000
- Bedrooms: 3
- Bathrooms: 2
- House Size: 1,750 sqft

**Description:** Step into this charming 3-bedroom, 2-bathroom home located in the desirable Richmond neighborhood. The house features a cozy living room, a well-appointed kitchen with stainless steel appliances, and a private backyard perfect for outdoor gatherings. With ample natural light, hardwood floors, and a spacious layout, this home is ideal for a growing family or those looking to downsize without compromising on comfort.

**Neighborhood Description:** Richmond is a vibrant neighborhood known for its blend of historic charm and modern convenience. Enjoy easy access to local shops, restaurants, and parks. Explore the nearby Richmond Art District or take a leisurely walk along the scenic riverfront. With top-rated schools and a strong sense of community, Richmond offers a welcoming atmosphere for residents of all ages.

**Persona:** The bu


> Finished chain.
**Listing Augmented:**

- Neighborhood: Notting Hill
- Price: $650,000
- Bedrooms: 4
- Bathrooms: 3
- House Size: 2025 sqft

**Description:** Step into luxury living in the heart of coveted Notting Hill. This stunning 4-bedroom, 3-bathroom home exudes elegance with its modern architecture and high-end finishes. The spacious interior is perfect for entertaining, featuring a gourmet kitchen, a cozy fireplace, and a master suite with a spa-like bathroom. Enjoy the tranquility of the beautifully landscaped backyard, ideal for outdoor gatherings or simply unwinding in the peaceful surroundings. Experience upscale living at its finest in this Notting Hill gem.

**Neighborhood Description:** Notting Hill is a prestigious neighborhood known for its tree-lined streets, upscale boutiques, and chic cafes. Residents enjoy easy access to the vibrant town center, renowned for its cultural events and world-class dining options. With top-rated schools and lush parks nearby, Notting 


> Finished chain.
**Listing 1 Augmented:**
- Neighborhood: Covent Garden
- Price: $425,000
- Bedrooms: 2
- Bathrooms: 2
- House Size: 1,250 sqft

**Description:** Step into this elegant 2-bedroom, 2-bathroom sanctuary located in the sought-after Covent Garden neighborhood. This meticulously maintained home features modern finishes throughout, including a gourmet kitchen with stainless steel appliances and granite countertops. The spacious living area opens up to a private balcony, perfect for enjoying your morning coffee or unwinding in the evenings. With its prime location and upscale amenities, this Covent Garden residence offers a luxurious lifestyle in the heart of the city.

**Neighborhood Description:** Covent Garden is a vibrant urban neighborhood known for its trendy boutiques, fine dining restaurants, and cultural attractions. Stroll along the bustling streets lined with historic architecture, or explore the nearby Covent Garden Market for fresh produce and artisanal goods. E


> Finished chain.
**Listing Augmented:**

- Neighborhood: Ealing
- Price: $350,000
- Bedrooms: 2
- Bathrooms: 2
- House Size: 1,150 sqft

**Description:** Welcome to this beautifully appointed 2-bedroom, 2-bathroom home in the desirable neighborhood of Ealing. The open floor plan offers a cozy living area, a modern kitchen with stainless steel appliances, and a spacious master suite. Enjoy the private patio overlooking the landscaped garden, perfect for relaxing with your morning coffee. This move-in ready home is ideal for those seeking comfort and style in a peaceful setting.

**Persona:** The buyer persona for this augmented listing is someone who values comfort and style in a peaceful environment. They prioritize a modern kitchen, a cozy living area, and a private outdoor space for relaxation, which influenced the choice of amenities and description in the listing. The persona seeks a move-in ready home in a charming and family-friendly neighborhood like Ealing, aligning with thei